Razlicni grafi za seminarsko 2022
Za pretvorbo med alkoholom v izdihanem zraku in v krvi se uporablja 2,083.
https://www.utoledo.edu/studentaffairs/counseling/selfhelp/substanceuse/bac.html, ponuja vrednost 1 : 2100, vendar je legalna meja 0,5 g/kg krvi oziroma 0,24 mg/l, v izdihanem zraku. Koeficient je torej 2.083.


In [3]:
#init code
import numpy as np
import matplotlib.pyplot as plt
from csv import DictReader

data = []
for year in range(2011, 2021):
    data += list(DictReader(open(f'../data/pn{year}.csv', 'rt', encoding='utf-8'), delimiter=";"))

for row in data:
    row["VrednostAlkotesta"] = row["VrednostAlkotesta"].replace(",", ".")
    row["VrednostStrokovnegaPregleda"] = row["VrednostStrokovnegaPregleda"].replace(",", ".")


def create_geo_array():
    latitudes = []
    longitudes = []
    for row in data:
        latitude = float(row["GeoKoordinataX"])
        longitude = float(row["GeoKoordinataY"])
        if (longitude == 0.0 or latitude == 0.0):
            continue
        latitudes.append(float(row["GeoKoordinataX"]))
        longitudes.append(float(row["GeoKoordinataY"]))
    return np.array([latitudes, longitudes])


df_geo = create_geo_array()

def qulify_accident(amounts_dict, key, row):
    item = amounts_dict[key]
    amounts_dict[key] = (item[0] + 1, item[1], item[2])
    if (row["KlasifikacijaNesrece"] == "S SMRTNIM IZIDOM"):
        amounts_dict[key] = (item[0], item[1], item[2] + 1)
    elif (row["KlasifikacijaNesrece"] == "S HUDO TELESNO POŠKODBO"):
        amounts_dict[key] = (item[0], item[1] + 1, item[2])
    return amounts_dict


def accidents_by_alcohol_amount():
    amounts_dict = dict()
    for row in data:
        a_alc = float(row["VrednostStrokovnegaPregleda"])
        if (a_alc == 0):
            a_alc = float(row["VrednostAlkotesta"]) * 2.083
        if (a_alc == 0):
            a_alc_class = 0
        elif (a_alc < 0.5):
             a_alc_class = 0.5
        elif (a_alc < 1):
            a_alc_class = 1
        elif (a_alc < 1.5):
            a_alc_class = 1.5
        elif (a_alc < 2):
            a_alc_class = 2
        elif (a_alc < 2.5):
            a_alc_class = 2.5
        else:
            a_alc_class = 3
        if amounts_dict.get(a_alc_class) is not None:
            amounts_dict = qulify_accident(amounts_dict, a_alc_class, row)
        else:
            amounts_dict[a_alc_class] = (0, 0, 0)
            amounts_dict = qulify_accident(amounts_dict, a_alc_class, row)
    return amounts_dict

alcohol_to_type = accidents_by_alcohol_amount()

/Users/erik/Documents/Faks/2.letnik/PR/projektna naloga/PR21LPTKRSTPEP/jupyter


In [ ]:
count_above_proposed = 0
count_above_current = 0
count_above_max = 0
count_alcoholized = 0
for row in data:
    amount_of_alcohol = float(row["VrednostStrokovnegaPregleda"])
    if (amount_of_alcohol == 0):
        amount_of_alchocol = float(row["VrednostAlkotesta"]) * 2.083
    if (amount_of_alcohol > 0):
        count_alcoholized += 1
    if amount_of_alcohol > 0.2:
        count_above_proposed += 1
    if amount_of_alcohol > 0.5:
        count_above_current += 1
    if amount_of_alcohol > 1.1:
        count_above_max += 1
print(
    f"Število alkoholiziranih: {count_alcoholized}, \n"
    f"nad 0.2: {count_above_proposed}, \n"
    f"nad 0.5: {count_above_current}, \n"
    f"nad 1.1: {count_above_max}, \n"
    f"število nesreč: {len(data)}")

In [ ]:
#Geolocational graph

BBox = (df_geo[1].min(), df_geo[1].max(), df_geo[0].min(), df_geo[0].max())
slo_map = plt.imread('data/map.png')

fig, ax = plt.subplots(figsize=(9, 8))
_ = ax.scatter(df_geo[1], df_geo[0], zorder=1, alpha=0.2, c='b', s=1)

ax.set_title('Lokacije nesreč v Sloveniji')
ax.set_xlim(BBox[0], BBox[1])
ax.set_ylim(BBox[2], BBox[3])
_ = ax.imshow(slo_map, zorder=0, extent=BBox, aspect='equal')

In [ ]:
#Povezanost med vsebnostjo alkohola in nesrečami s težkimi poškodbami in smrtmi
alcohol_amounts = sorted(alcohol_to_type.keys())
ratios_injury_death = []
ratios_death = []
for key in alcohol_amounts:
    item = alcohol_to_type[key]
    try:
        ratios_injury_death.append((item[1] + item[2]) / item[0])
        ratios_death.append((item[2]) / item[0])
    except ZeroDivisionError:
        pass

fig, ax = plt.subplots(1, 2, figsize=(9, 3))
ax[0].plot(alcohol_amounts, ratios_injury_death ,c='r')

ax[1].plot(alcohol_amounts, ratios_death ,c='r')

ax[0].set_title('Vsebnost alkohola in procent nesreč,\n pri katerih je prišlo do težjih poškodb ali smrti')
ax[1].set_title("Vsebnost alkohola in procent nesreč,\n pri katerih je prišlo do smrti")

